## Example submission

Image Matching Challenge 2025: https://www.kaggle.com/competitions/image-matching-challenge-2025

This notebook creates a simple submission using ALIKED and LightGlue, plus DINO for shortlisting, on GPU. Adapted from [last year](https://www.kaggle.com/code/oldufo/imc-2024-submission-example).

Remember to select an accelerator on the sidebar to the right, and to disable internet access when submitting a notebook to the competition.

In [5]:
# IMPORTANT 
#Install dependencies and copy model weights to run the notebook without internet access when submitting to the competition.

!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/aliked-n16.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
kornia is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
kornia-moons is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
kornia-rs is already installed 

In [6]:
!cp /kaggle/input/disk-depth/disk_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/disk-depth/disk_lightglue.pth /root/.cache/torch/hub/checkpoints/disk_lightglue_v0-1_arxiv-pth
!cp /kaggle/input/disk-depth/depth-save.pth /root/.cache/torch/hub/checkpoints/depth-save.pth
!cp /kaggle/input/disk-depth/depth-save.pth /root/.cache/torch/hub/checkpoints/

In [7]:
!cp /kaggle/input/superpoint-lightglue/superpoint_lightglue.pth /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/superpoint-lightglue/superpoint_lightglue.pth /root/.cache/torch/hub/checkpoints//root/.cache/torch/hub/checkpoints/superpoint_lightglue_v0-1_arxiv-pth
!cp /kaggle/input/superpoint-lightglue/superpoint_v1.pth /root/.cache/torch/hub/checkpoints/superpoint_v1.pth
!cp /kaggle/input/superpoint-lightglue/superpoint_v1.pth /root/.cache/torch/hub/checkpoints/

cp: cannot create regular file '/root/.cache/torch/hub/checkpoints//root/.cache/torch/hub/checkpoints/superpoint_lightglue_v0-1_arxiv-pth': No such file or directory


In [8]:
import os
print(os.path.exists("/root/.cache/torch/hub/checkpoints/depth-save.pth"))

True


In [9]:
import sys
import os
from tqdm import tqdm
from time import time, sleep
import gc
import numpy as np
import h5py
import dataclasses
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from PIL import Image

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

import torch
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd
from transformers import AutoImageProcessor, AutoModel

# from lightglue import DISK
from kornia.feature import LightGlueMatcher as KF_LightGlueMatcher
from scipy.spatial import cKDTree # For efficient nearest neighbor search to remove duplicate keypoints

# IMPORTANT Utilities: importing data into colmap and competition metric
import pycolmap
sys.path.append('/kaggle/input/imc25-utils')
from database import *
from h5_to_db import *
import metric


# LightGlue
from lightglue import match_pair
from lightglue import ALIKED, SuperPoint,DISK, DoGHardNet, LightGlue, SIFT
from fastprogress import progress_bar


/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/usr/local/lib/python3.10/dist-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [10]:
from collections import defaultdict
from copy import deepcopy
import concurrent.futures


In [11]:

print("PyTorch version:", torch.__version__)
import sys
print("Python version:", sys.version)

print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


PyTorch version: 2.5.1+cu121
Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
CUDA available: True
CUDA version: 12.1
Device count: 2
Current device: 0
Device name: Tesla T4


In [12]:
# Do not forget to select an accelerator on the sidebar to the right.
device = K.utils.get_cuda_device_if_available(0)
print(f'{device=}')

device=device(type='cuda', index=0)


In [13]:
VERBOSE = True

In [82]:
class CONFIG:
    # DEBUG Settings
    DRY_RUN = False
    DRY_RUN_MAX_IMAGES = 10

    # Pipeline settings
    NUM_CORES = 2
    
    # COLMAP Reconstruction
    CAMERA_MODEL = "simple-radial"
    
    # Rotation correction
    ROTATION_CORRECTION = False
    
    # Keypoints handling
    MERGE_PARAMS = {
        "min_matches" : 15,
        # When merging keypoints, it is enable to filtering matches with cv2.findFundamentalMatrix.
        "filter_FundamentalMatrix" : True,
        "filter_iterations" : 5,
        "filter_threshold" : 3,
    }
    
    # Keypoints Extraction
    use_aliked_lightglue = True
    use_doghardnet_lightglue = False
    use_superpoint_lightglue = True
    use_disk_lightglue = True
    use_sift_lightglue = False
    use_loftr = False
    use_dkm = False
    use_superglue = False
    use_matchformer = False
        
    # Keypoints Extraction Parameters
    params_aliked_lightglue = {
        "num_features" : 4096,
        "detection_threshold" : 0.1,
        "min_matches" : 50,
        "resize_to" : 2048,
        "match_confidence_threshold":1.0
    }
    
    params_doghardnet_lightglue = {
        "num_features" : 8192,
        "detection_threshold" : 0.001,
        "min_matches" : 15,
        "resize_to" : 1024,
    }
    
    params_superpoint_lightglue = {
        "num_features" : 4096,
        "detection_threshold" : 0.1,
        "min_matches" : 50,
        "resize_to" : 1024,
    }
    
    params_disk_lightglue = {
        "num_features" : 4096,
        "detection_threshold" : 0.1,
        "min_matches" : 50,
        "resize_to" : 2048,
        "match_confidence_threshold":1.0
    }

    params_sift_lightglue = {
        "num_features" : 8192,
        "detection_threshold" : 0.001,
        "min_matches" : 15,
        "resize_to" : 1024,
    }

    params_loftr = {
        "resize_small_edge_to" : 750,
        "min_matches" : 15,
    }
    
    params_dkm = {
        "num_features" : 2048,
        "detection_threshold" : 0.4,
        "min_matches" : 15,
        "resize_to" : (540, 720),    
    }
    
    # superpoint + superglue  ...  https://www.kaggle.com/competitions/image-matching-challenge-2023/discussion/416873
    params_sg1 = {
        "sg_config" : 
        {
            "superpoint": {
                "nms_radius": 4, 
                "keypoint_threshold": 0.005,
                "max_keypoints": -1,
            },
            "superglue": {
                "weights": "outdoor",
                "sinkhorn_iterations": 20,
                "match_threshold": 0.2,
            },
        },
        "resize_to": 1088,
        "min_matches": 15,
    }
    params_sg2 = {
        "sg_config" : 
        {
            "superpoint": {
                "nms_radius": 4, 
                "keypoint_threshold": 0.005,
                "max_keypoints": -1,
            },
            "superglue": {
                "weights": "outdoor",
                "sinkhorn_iterations": 20,
                "match_threshold": 0.2,
            },
        },
        "resize_to": 1280,
        "min_matches": 15,
    }
    params_sg3 = {
        "sg_config" : 
        {
            "superpoint": {
                "nms_radius": 4, 
                "keypoint_threshold": 0.005,
                "max_keypoints": -1,
            },
            "superglue": {
                "weights": "outdoor",
                "sinkhorn_iterations": 20,
                "match_threshold": 0.2,
            },
        },
        "resize_to": 1376,
        "min_matches": 15,
    }
    params_sgs = [params_sg1, params_sg2, params_sg3]
    
    params_matchformer = {
        "detection_threshold" : 0.15,
        "resize_to" : (560, 750),
        "num_features" : 2000,
        "min_matches" : 15, 
    }

In [83]:
# Assume these are available from your environment or previous code
# from .utils import load_torch_image # Assuming load_torch_image is defined elsewhere
# from kornia.feature import ALIKED # Already in your detect_aliked
# from kornia.feature import LightGlueMatcher as KF_LightGlueMatcher # Already in your match_with_lightglue
# from kornia.geometry import laf_from_center_scale_ori # Already in your match_with_lightglue
# from colmap_database import COLMAPDatabase, add_keypoints, add_matches # Already in your colmap_import

# --- Helper function for image loading (if not already defined) ---
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img
def get_dino_patch_features_for_keypoints(img_path, keypoints_xy, dino_processor, dino_model, patch_size=16, device=torch.device('cpu')):
    """
    Extracts DINO patch features corresponding to given ALIKED keypoint locations.
    It correctly infers the DINO patch grid dimensions from the processed input.

    Args:
        img_path (str): Path to the image file.
        keypoints_xy (torch.Tensor): Nx2 tensor of (x, y) keypoint coordinates in image pixel space.
                                     These keypoints are assumed to be in the original image's coordinate system.
        dino_processor: HuggingFace AutoImageProcessor for DINO.
        dino_model: HuggingFace AutoModel for DINO.
        patch_size (int): The patch size used by the DINO model (e.g., 14 or 16).
        device (torch.device): Device to run the models on.

    Returns:
        torch.Tensor: NxD_dino tensor of DINO patch features for each keypoint.
                      Returns None if no keypoints or image loading fails.
    """
    if len(keypoints_xy) == 0:
        dino_feature_dim = dino_model.config.hidden_size # Get actual DINO hidden size
        return torch.empty((0, dino_feature_dim), device=device)

    # 1. Load the original image (ALIKED processed this size)
    original_img = load_torch_image(img_path, device=device)
    original_h, original_w = original_img.shape[-2], original_img.shape[-1]


    # 2. Process the image with DINO's processor
    #    This step performs resizing, padding, etc., as needed by the DINO model
    with torch.inference_mode():
        # dino_processor returns a BatchFeature object which includes pixel_values
        # and potentially other information like `pixel_mask`
        inputs = dino_processor(images=original_img, return_tensors="pt", do_rescale=False).to(device)
        outputs = dino_model(**inputs)

        # Get the actual dimensions of the image as processed by the DINO model
        # This is the crucial part: the actual H and W that produced `patch_tokens`
        # We can infer this from the `pixel_values` shape
        processed_h = inputs['pixel_values'].shape[-2]
        processed_w = inputs['pixel_values'].shape[-1]

        # Extract patch tokens (excluding the CLS token)
        patch_tokens = outputs.last_hidden_state[:, 1:].squeeze(0) # Shape: (num_patches, hidden_size)

        # Calculate the actual grid dimensions based on the *processed* image size
        # and the model's patch size.
        # This should perfectly match the number of patch_tokens if the model is well-behaved.
        num_patches_h = processed_h // patch_size
        num_patches_w = processed_w // patch_size

        # Safety check: ensure calculated grid matches actual token count
        expected_token_count = num_patches_h * num_patches_w
        if patch_tokens.shape[0] != expected_token_count:
            # This indicates a deeper issue with how the model's output tokens
            # map to the spatial grid, or an unexpected patch size/model behavior.
            # Some models might have slightly different patch token arrangements.
            # DINOv2 typically aligns well.
            raise ValueError(
                f"DINO patch token count ({patch_tokens.shape[0]}) does not match "
                f"expected grid dimensions ({num_patches_h}x{num_patches_w} = {expected_token_count}) "
                f"for processed image size {processed_w}x{processed_h} with patch size {patch_size}. "
                f"Please verify DINO model and processor configuration."
            )

        # Reshape patch tokens into a 2D grid
        patch_features_grid = patch_tokens.reshape(num_patches_h, num_patches_w, -1)
        dino_feature_dim = patch_features_grid.shape[-1] # Actual feature dimension


    dino_features_for_kpts = torch.zeros((len(keypoints_xy), dino_feature_dim), device=device)

    # 3. Rescale ALIKED keypoints to the DINO *processed* image dimensions
    #    ALIKED keypoints are in original_w x original_h coordinates.
    #    DINO patches correspond to processed_w x processed_h coordinates.
    scale_x = processed_w / original_w
    scale_y = processed_h / original_h

    scaled_keypoints_xy = keypoints_xy.clone()
    scaled_keypoints_xy[:, 0] *= scale_x
    scaled_keypoints_xy[:, 1] *= scale_y

    # 4. Map scaled keypoints to DINO patch grid indices
    keypoint_cols = (scaled_keypoints_xy[:, 0] / patch_size).long()
    keypoint_rows = (scaled_keypoints_xy[:, 1] / patch_size).long()

    # Clip indices to ensure they are within bounds of the patch grid
    keypoint_rows = torch.clamp(keypoint_rows, 0, num_patches_h - 1)
    keypoint_cols = torch.clamp(keypoint_cols, 0, num_patches_w - 1)

    # Gather DINO features for each keypoint's corresponding patch
    dino_features_for_kpts = patch_features_grid[keypoint_rows, keypoint_cols]

    return dino_features_for_kpts


def convert_coord(r, w, h, rotk):
    if rotk == 0:
        return r
    elif rotk == 1:
        rx = w-1-r[:, 1]
        ry = r[:, 0]
        return torch.concat([rx[None], ry[None]], dim=0).T
    elif rotk == 2:
        rx = w-1-r[:, 0]
        ry = h-1-r[:, 1]
        return torch.concat([rx[None], ry[None]], dim=0).T
    elif rotk == 3:
        rx = r[:, 1]
        ry = h-1-r[:, 0]
        return torch.concat([rx[None], ry[None]], dim=0).T

def detect_common(img_fnames,
                  model_name,
                  rots,
                  file_keypoints,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  detection_threshold = 0.01,
                  device=torch.device('cpu'),
                  min_matches=15,
                  match_confidence_threshold = 0.0,
                  verbose=VERBOSE
                 ):
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)

    #####################################################
    # Extract keypoints and descriptions
    #####################################################
    dict_model = {
        "aliked" : ALIKED,
        "superpoint" : SuperPoint,
        "doghardnet" : DoGHardNet,
        "disk" : DISK,
        "sift" : SIFT,
    }
    extractor_class = dict_model[model_name]
    dtype = torch.float32 # ALIKED has issues with float16
    # extractor = extractor_class(max_num_keypoints=num_features, detection_threshold=detection_threshold, 
    #                             resize=resize_to).eval().to(device, dtype)
    # if model_name == 'disk':
    #     extractor = DISK(
    #         max_num_keypoints=num_features,
    #         detection_threshold=detection_threshold,
    #         resize=resize_to
    #     ).to(device).eval()
    #     checkpoint = torch.load(ckpt_path, map_location=device)
    #     extractor.load_state_dict(checkpoint['model'])
    # else:
    #     extractor_class = dict_model[model_name]
    #     extractor = extractor_class(
    #         max_num_keypoints=num_features,
    #         detection_threshold=detection_threshold,
    #         resize=resize_to
    #     ).to(device, dtype).eval()

    extractor_class = dict_model[model_name]
    extractor = extractor_class(
        max_num_keypoints=num_features,
        detection_threshold=detection_threshold,
        resize=resize_to
    ).to(device, dtype).eval()
    dict_kpts_cuda = {}
    dict_descs_cuda = {}
    for (img_path, rot_k) in zip(img_fnames, rots):
        img_fname = img_path.split('/')[-1]
        key = img_fname
        with torch.inference_mode():
            image0 = load_torch_image(img_path, device=device).to(dtype)
            h, w = image0.shape[2], image0.shape[3]
            image1 = torch.rot90(image0, rot_k, [2, 3])
            feats0 = extractor.extract(image1)  # auto-resize the image, disable with resize=None
            kpts = feats0['keypoints'].reshape(-1, 2).detach()
            descs = feats0['descriptors'].reshape(len(kpts), -1).detach()
            kpts = convert_coord(kpts, w, h, rot_k)
            dict_kpts_cuda[f"{key}"] = kpts
            dict_descs_cuda[f"{key}"] = descs
            if verbose:
                print(f"{model_name} > rot_k={rot_k}, kpts.shape={kpts.shape}, descs.shape={descs.shape}")
    del extractor
    gc.collect()

    #####################################################
    # Matching keypoints
    #####################################################
    lg_matcher = KF.LightGlueMatcher(model_name, {"width_confidence": -1,
                                            "depth_confidence": -1,
                                             "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    
    cnt_pairs = 0
    with h5py.File(file_keypoints, mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            
            kp1 = dict_kpts_cuda[key1]
            kp2 = dict_kpts_cuda[key2]
            desc1 = dict_descs_cuda[key1]
            desc2 = dict_descs_cuda[key2]
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                     desc2,
                                     KF.laf_from_center_scale_ori(kp1[None]),
                                     KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            len1 = len(idxs)
            n_matches = len1
            # if len(idxs) >= min_matches:                
            #     conf = dists.cpu().numpy()  # lower is better
            #     if conf.ndim == 2:
            #         conf = conf[:, 0]  # force (N,)
            #     conf_mask = conf <= match_confidence_threshold
            #     if not np.any(conf_mask):
            #         continue
            #     idxs = idxs[conf_mask]
            #     conf = conf[conf_mask]
            #     n_matches = len(idxs)
            #     if verbose:
            #         print(f"match after conf threshold: {key1}-{key2}: {len1}->{n_matches}")
            kp1 = kp1[idxs[:,0], :].cpu().numpy().reshape(-1, 2).astype(np.float32)
            kp2 = kp2[idxs[:,1], :].cpu().numpy().reshape(-1, 2).astype(np.float32)
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                group.create_dataset(key2, data=np.concatenate([kp1, kp2], axis=1))
                cnt_pairs+=1
                if verbose:
                    print (f'{model_name}> {key1}-{key2}: {n_matches} matches @ {cnt_pairs}th pair({model_name}+lightglue)')            
            else:
                pass
                # if verbose:
                #     print (f'{model_name}> {key1}-{key2}: {n_matches} matches --> skipped')
    del lg_matcher
    torch.cuda.empty_cache()
    gc.collect()
    return

def detect_lightglue_common(
    img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints, rots,
    resize_to=1024,
    detection_threshold=0.01, 
    num_features=4096, 
    min_matches=15,
    match_confidence_threshold = 0.0
):
    t=time()
    detect_common(
        img_fnames, model_name, rots, file_keypoints, feature_dir, 
        resize_to=resize_to,
        num_features=num_features, 
        detection_threshold=detection_threshold, 
        device=device,
        min_matches=min_matches,
        match_confidence_threshold = match_confidence_threshold
    )
    gc.collect()
    t=time() -t 
    print(f'Features matched in  {t:.4f} sec ({model_name}+LightGlue)')
    return t

In [84]:
def get_unique_idxs(A, dim=0):
    # https://stackoverflow.com/questions/72001505/how-to-get-unique-elements-and-their-firstly-appeared-indices-of-a-pytorch-tenso
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    return first_indices

def get_keypoint_from_h5(fp, key1, key2):
    rc = -1
    try:
        kpts = np.array(fp[key1][key2])
        rc = 0
        return (rc, kpts)
    except:
        return (rc, None)

def get_keypoint_from_multi_h5(fps, key1, key2):
    list_mkpts = []
    for fp in fps:
        rc, mkpts = get_keypoint_from_h5(fp, key1, key2)
        if rc == 0:
            list_mkpts.append(mkpts)
    if len(list_mkpts) > 0:
        list_mkpts = np.concatenate(list_mkpts, axis=0)
    else:
        list_mkpts = None
    return list_mkpts

def matches_merger(
    img_fnames,
    index_pairs,
    files_keypoints,
    save_file,
    feature_dir = 'featureout',
    filter_FundamentalMatrix = False,
    filter_iterations = 10,
    filter_threshold = 8,
    verbose = VERBOSE
):
    # open h5 files
    fps = [ h5py.File(file, mode="r") for file in files_keypoints ]

    with h5py.File(save_file, mode='w') as f_match:
        counter = 0
        for pair_idx in progress_bar(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]

            # extract keypoints
            mkpts = get_keypoint_from_multi_h5(fps, key1, key2)
            if mkpts is None:
                # if verbose:
                #     print(f"skipped key1={key1}, key2={key2}")
                continue

            ori_size = mkpts.shape[0]
            if mkpts.shape[0] < CONFIG.MERGE_PARAMS["min_matches"]:
                continue
            
            if filter_FundamentalMatrix:
                store_inliers = { idx:0 for idx in range(mkpts.shape[0]) }
                idxs = np.array(range(mkpts.shape[0]))
                for iter in range(filter_iterations):
                    try:
                        Fm, inliers = cv2.findFundamentalMat(
                            mkpts[:,:2], mkpts[:,2:4], cv2.USAC_MAGSAC, 0.15, 0.9999, 20000)
                        if Fm is not None:
                            inliers = inliers > 0
                            inlier_idxs = idxs[inliers[:, 0]]
                            #print(inliers.shape, inlier_idxs[:5])
                            for idx in inlier_idxs:
                                store_inliers[idx] += 1
                    except:
                        print(f"Failed to cv2.findFundamentalMat. mkpts.shape={mkpts.shape}")
                inliers = np.array([ count for (idx, count) in store_inliers.items() ]) >= filter_threshold
                mkpts = mkpts[inliers]
                if mkpts.shape[0] < 15:
                    if verbose:
                        print(f"skipped key1={key1}, key2={key2}: mkpts.shape={mkpts.shape} after filtered.")
                    continue
                print(f"filter_FundamentalMatrix: {len(store_inliers)} matches --> {mkpts.shape[0]} matches")
            
            if verbose:
                print (f'{key1}-{key2}: {ori_size} --> {mkpts.shape[0]} matches')            
            # regist tmp file
            group  = f_match.require_group(key1)
            group.create_dataset(key2, data=mkpts)
            counter += 1
    print( f"Ensembled pairs : {counter} pairs" )
    for fp in fps:
        fp.close()

def keypoints_merger(
    img_fnames,
    index_pairs,
    files_keypoints,
    feature_dir = 'featureout',
    filter_FundamentalMatrix = False,
    filter_iterations = 10,
    filter_threshold = 8,
):
    save_file = f'{feature_dir}/merge_tmp.h5'
    !rm -rf {save_file}
    matches_merger(
        img_fnames,
        index_pairs,
        files_keypoints,
        save_file,
        feature_dir = feature_dir,
        filter_FundamentalMatrix = filter_FundamentalMatrix,
        filter_iterations = filter_iterations,
        filter_threshold = filter_threshold,
    )
        
    # Let's find unique loftr pixels and group them together.
    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    with h5py.File(save_file, mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0]+=total_kpts[k1]
                current_match[:, 1]+=total_kpts[k2]
                total_kpts[k1]+=len(matches)
                total_kpts[k2]+=len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([unique_kpts[k1][ m2[:,0]],
                                    unique_kpts[k2][  m2[:,1]],
                                   ],
                                   axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1
    
    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
                # print(f"KKKKKKK KKKKKK {k1} - {k2}: {len(match)} matches")
    return

In [85]:
# !rm -rf /kaggle/working/result

In [86]:
# --- MODIFIED: Detect ALIKED and Combine with DINO Patch Features ---
def detect_aliked_and_combine_with_dino(img_fnames,
                                        feature_dir='.featureout',
                                        num_features=4096,
                                        resize_to=1024,
                                        dino_processor=None,
                                        dino_model=None,
                                        dino_patch_size=16, # Typically 14 or 16 for DINO
                                        device=torch.device('cpu')):
    dtype = torch.float32 # ALIKED has issues with float16
    aliked_extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.1).eval().to(device, dtype)
    aliked_extractor.preprocess_conf["resize"] = resize_to
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)

    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors_aliked.h5', mode='w') as f_desc_aliked, \
         h5py.File(f'{feature_dir}/descriptors_combined.h5', mode='w') as f_desc_combined: # New HDF5 for combined features
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname

            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = aliked_extractor.extract(image0)
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy() # ALIKED keypoints (x,y)
                descs_aliked = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy() # ALIKED descriptors

                # Get DINO patch features for these keypoints
                kpts_torch = torch.from_numpy(kpts).to(device)
                descs_dino_patch = get_dino_patch_features_for_keypoints(
                    img_path, kpts_torch, dino_processor, dino_model, dino_patch_size, device
                ).detach().cpu().numpy()

                # Concatenate ALIKED and DINO features
                if len(descs_aliked) > 0 and len(descs_dino_patch) > 0:
                    combined_descs = np.concatenate((descs_aliked, descs_dino_patch), axis=1)
                elif len(descs_aliked) > 0: # Only ALIKED if no DINO features (shouldn't happen often)
                    combined_descs = descs_aliked
                else: # No features found
                    combined_descs = np.array([]) # Empty array

                f_kp[key] = kpts
                f_desc_aliked[key] = descs_aliked # Keep ALIKED descriptors for debugging or other uses
                f_desc_combined[key] = combined_descs # Store the new combined descriptors
    print(f"Combined features saved to {feature_dir}/descriptors_combined.h5")
    return

In [87]:
from sklearn.cluster import MiniBatchKMeans # MiniBatchKMeans is faster for large datasets

# --- VLAD Aggregation Function ---
def vlad_encode(descriptors, centroids):
    """
    Performs VLAD encoding.

    Args:
        descriptors (np.ndarray): NxM array of local descriptors.
        centroids (np.ndarray): KxM array of K-Means cluster centroids.

    Returns:
        np.ndarray: 1x(K*M) VLAD descriptor.
    """
    if descriptors.shape[0] == 0:
        return np.zeros(centroids.shape[0] * centroids.shape[1], dtype=np.float32)

    num_descriptors, desc_dim = descriptors.shape
    num_centroids, _ = centroids.shape

    # Assign each descriptor to its nearest centroid
    # Using cdist for efficiency
    distances = np.sqrt(np.sum((descriptors[:, None, :] - centroids[None, :, :])**2, axis=2))
    # distances = cdist(descriptors, centroids, 'sqeuclidean') # Could use cdist for sqeuclidean
    cluster_assignments = np.argmin(distances, axis=1)

    # Initialize VLAD accumulator
    vlad_accumulator = np.zeros((num_centroids, desc_dim), dtype=np.float32)

    # Accumulate residuals
    for i in range(num_descriptors):
        cluster_idx = cluster_assignments[i]
        residual = descriptors[i] - centroids[cluster_idx]
        vlad_accumulator[cluster_idx] += residual

    # Flatten and L2 normalize
    vlad_descriptor = vlad_accumulator.flatten()
    vlad_descriptor = F.normalize(torch.from_numpy(vlad_descriptor).unsqueeze(0), dim=1, p=2).squeeze(0).numpy()

    return vlad_descriptor

In [88]:
# --- NEW: Get Global Descriptors using K-Means + VLAD ---
def get_global_desc_vlad(fnames, feature_dir='.featureout', num_clusters=64, device=torch.device('cpu')):
    """
    Generates global descriptors for images using K-Means + VLAD on combined ALIKED+DINO features.

    Args:
        fnames (list): List of image file paths.
        feature_dir (str): Directory where combined descriptors are stored.
        num_clusters (int): Number of clusters for K-Means (K in VLAD).
        device (torch.device): Not directly used for VLAD computation, but passed for consistency.

    Returns:
        torch.Tensor: Nx(K*M) tensor of global VLAD descriptors.
    """
    all_local_descs = []
    keys_order = [] # To maintain order of descriptors with respect to fnames

    # 1. Load all combined local descriptors
    with h5py.File(f'{feature_dir}/descriptors_combined.h5', mode='r') as f_desc_combined:
        for img_path in tqdm(fnames, desc="Loading combined local descriptors for K-Means"):
            key = img_path.split('/')[-1]
            if key in f_desc_combined:
                descs = f_desc_combined[key][...]
                if descs.shape[0] > 0:
                    all_local_descs.append(descs)
                    keys_order.append(key)

    if not all_local_descs:
        print("No combined local descriptors found. Cannot train K-Means or compute VLAD.")
        return torch.empty((0, num_clusters * 0), dtype=torch.float32) # Return empty tensor

    # Concatenate all descriptors for K-Means training
    all_local_descs_flat = np.concatenate(all_local_descs, axis=0)

    # 2. Train K-Means on a subset of descriptors if the dataset is too large
    # Or directly on all_local_descs_flat if memory permits
    print(f"Training K-Means with {num_clusters} clusters on {all_local_descs_flat.shape[0]} descriptors...")
    # Use MiniBatchKMeans for efficiency
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=0, n_init='auto', batch_size=256).fit(all_local_descs_flat)
    centroids = kmeans.cluster_centers_
    print("K-Means training complete.")

    # 3. Compute VLAD descriptor for each image
    global_descs_vlad = []
    # Re-iterate through original fnames to match the output order
    with h5py.File(f'{feature_dir}/descriptors_combined.h5', mode='r') as f_desc_combined:
        for img_path in tqdm(fnames, desc="Computing VLAD descriptors"):
            key = img_path.split('/')[-1]
            if key in f_desc_combined:
                descs = f_desc_combined[key][...]
                vlad_desc = vlad_encode(descs, centroids)
                global_descs_vlad.append(torch.from_numpy(vlad_desc).unsqueeze(0))
            else:
                # Handle cases where an image might not have any combined descriptors
                # (e.g., no ALIKED keypoints detected). Append a zero vector of correct size.
                print(f"Warning: No combined descriptors for {key}. Appending zero VLAD descriptor.")
                # Determine descriptor dimension from centroids
                desc_dim_per_cluster = centroids.shape[1] if centroids.shape[1] > 0 else 0 # Should not be 0 normally
                zero_vlad = np.zeros(num_clusters * desc_dim_per_cluster, dtype=np.float32)
                global_descs_vlad.append(torch.from_numpy(zero_vlad).unsqueeze(0))


    if not global_descs_vlad:
        return torch.empty((0, num_clusters * centroids.shape[1] if centroids.shape[1] > 0 else 0), dtype=torch.float32)

    global_descs_vlad = torch.cat(global_descs_vlad, dim=0)
    return global_descs_vlad

In [89]:
# --- RE-DEFINED: get_image_pairs_shortlist to use the new VLAD global descriptor ---
def get_image_pairs_shortlist_vlad(fnames,
                                   sim_th=0.6, # should be strict
                                   min_pairs=30,
                                   exhaustive_if_less=20,
                                   feature_dir='.featureout', # Pass feature_dir
                                   num_clusters_vlad=64, # New parameter for VLAD
                                   device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames) # You need to define get_img_pairs_exhaustive if not done.

    # Use the new VLAD-based global descriptor
    descs = get_global_desc_vlad(fnames, feature_dir=feature_dir, num_clusters=num_clusters_vlad, device=device)

    if descs.shape[0] == 0:
        print("No global descriptors generated. Returning empty matching list.")
        return []

    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()

    # 只分析上三角（去掉对角线），避免重复
    triu_indices = np.triu_indices_from(dm, k=1)
    dm_flat = dm[triu_indices]
    
    # 打印统计信息
    print("Distance Matrix Statistics:")
    print(f"Min:  {dm_flat.min():.4f}")
    print(f"Max:  {dm_flat.max():.4f}")
    print(f"Mean: {dm_flat.mean():.4f}")
    print(f"Std:  {dm_flat.std():.4f}")
    print(f"20%:  {np.percentile(dm_flat, 20):.4f}")
    print(f"25%:  {np.percentile(dm_flat, 25):.4f}")
    print(f"USED 60%:  {np.percentile(dm_flat, 60):.4f}")
    print(f"75%:  {np.percentile(dm_flat, 75):.4f}")
    threshold = np.percentile(dm_flat, 60) + np.sqrt(3) * dm_flat.std()

    # removing half
    mask = dm <= np.percentile(dm_flat, 60)
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = set() # Use a set for faster lookup of already added pairs

    for st_idx in range(num_imgs - 1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]

        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < threshold: # Ensure distance is not effectively infinite
                pair = tuple(sorted((st_idx, idx.item())))
                if pair not in already_there_set:
                    matching_list.append(pair)
                    already_there_set.add(pair)
                    total += 1
    matching_list = sorted(list(matching_list)) # Sort the list of tuples
    return matching_list

In [90]:
def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs

In [91]:
# Must Use efficientnet global descriptor to get matching shortlists.
def get_global_desc(fnames, device = torch.device('cpu')):
    processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = model.eval()
    model = model.to(device)
    global_descs_dinov2 = []
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        timg = load_torch_image(img_fname_full)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_mac = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2)
        global_descs_dinov2.append(dino_mac.detach().cpu())
    global_descs_dinov2 = torch.cat(global_descs_dinov2, dim=0)
    return global_descs_dinov2


def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


def get_image_pairs_shortlist(fnames,
                              sim_th=0.6,
                              min_pairs=30,
                              max_pairs=100,  # 每张图像最多匹配 max_pairs 个
                              exhaustive_if_less=20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)

    descs = get_global_desc(fnames, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()

    # 上三角分析（排除重复）
    triu_indices = np.triu_indices_from(dm, k=1)
    dm_flat = dm[triu_indices]

    print("Distance Matrix Statistics:")
    print(f"Min:  {dm_flat.min():.4f}")
    print(f"Max:  {dm_flat.max():.4f}")
    print(f"Mean: {dm_flat.mean():.4f}")
    print(f"Std:  {dm_flat.std():.4f}")
    print(f"20%:  {np.percentile(dm_flat, 20):.4f}")
    print(f"25%:  {np.percentile(dm_flat, 25):.4f}")
    print(f"60%:  {np.percentile(dm_flat, 60):.4f}")
    print(f"75%:  {np.percentile(dm_flat, 75):.4f}")

    threshold = np.percentile(dm_flat, 60) + np.sqrt(3) * dm_flat.std()
    mask = dm <= np.percentile(dm_flat, 50)

    ar = np.arange(num_imgs)
    matching_set = set()

    for st_idx in range(num_imgs):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]

        # 保证每张图像至少有 min_pairs 个
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]

        # 按距离排序，选出前 max_pairs
        sorted_matches = sorted(
            [(idx, dm[st_idx, idx]) for idx in to_match if idx != st_idx and dm[st_idx, idx] < threshold],
            key=lambda x: x[1]
        )
        for idx, _ in sorted_matches[:max_pairs]:
            pair = tuple(sorted((st_idx, idx)))
            matching_set.add(pair)

    matching_list = sorted(list(matching_set))
    return matching_list


In [92]:
def wrapper_keypoints(
    img_fnames, index_pairs, feature_dir, device, timings, rots
):
    #############################################################
    # get keypoints
    #############################################################
    files_keypoints = []
    
    if CONFIG.use_superglue:
        for params_sg in CONFIG.params_sgs:
            resize_to = params_sg["resize_to"]
            file_keypoints = f"{feature_dir}/matches_superglue_{resize_to}pix.h5"
            !rm -rf {file_keypoints}
            t = detect_superglue(
                img_fnames, index_pairs, feature_dir, device, 
                params_sg["sg_config"], file_keypoints, 
                resize_to=params_sg["resize_to"], 
                min_matches=params_sg["min_matches"],
            )
            gc.collect()
            files_keypoints.append( file_keypoints )
            timings['feature_matching'].append(t)

    if CONFIG.use_aliked_lightglue:
        model_name = "aliked"
        file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
        t = detect_lightglue_common(
            img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints, rots,
            resize_to=CONFIG.params_aliked_lightglue["resize_to"],
            detection_threshold=CONFIG.params_aliked_lightglue["detection_threshold"],
            num_features=CONFIG.params_aliked_lightglue["num_features"],
            min_matches=CONFIG.params_aliked_lightglue["min_matches"],
            match_confidence_threshold=CONFIG.params_aliked_lightglue["match_confidence_threshold"]
        )
        gc.collect()
        files_keypoints.append(file_keypoints)
        timings['feature_matching'].append(t)

    if CONFIG.use_doghardnet_lightglue:
        model_name = "doghardnet"
        file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
        t = detect_lightglue_common(
            img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints, rots,
            resize_to=CONFIG.params_doghardnet_lightglue["resize_to"],
            detection_threshold=CONFIG.params_doghardnet_lightglue["detection_threshold"],
            num_features=CONFIG.params_doghardnet_lightglue["num_features"],
            min_matches=CONFIG.params_doghardnet_lightglue["min_matches"],
        )
        gc.collect()
        files_keypoints.append(file_keypoints)
        timings['feature_matching'].append(t)

    if CONFIG.use_superpoint_lightglue:
        model_name = "superpoint"
        file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
        t = detect_lightglue_common(
            img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints, rots,
            resize_to=CONFIG.params_superpoint_lightglue["resize_to"],
            detection_threshold=CONFIG.params_superpoint_lightglue["detection_threshold"],
            num_features=CONFIG.params_superpoint_lightglue["num_features"],
            min_matches=CONFIG.params_superpoint_lightglue["min_matches"],
        )
        gc.collect()
        files_keypoints.append(file_keypoints)
        timings['feature_matching'].append(t)

    if CONFIG.use_disk_lightglue:
        model_name = "disk"
        file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
        t = detect_lightglue_common(
            img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints, rots,
            resize_to=CONFIG.params_disk_lightglue["resize_to"],
            detection_threshold=CONFIG.params_disk_lightglue["detection_threshold"],
            num_features=CONFIG.params_disk_lightglue["num_features"],
            min_matches=CONFIG.params_disk_lightglue["min_matches"],
            match_confidence_threshold=CONFIG.params_disk_lightglue["match_confidence_threshold"]
        )
        gc.collect()
        files_keypoints.append(file_keypoints)
        timings['feature_matching'].append(t)

    if CONFIG.use_sift_lightglue:
        model_name = "sift"
        file_keypoints = f'{feature_dir}/matches_lightglue_{model_name}.h5'
        t = detect_lightglue_common(
            img_fnames, model_name, index_pairs, feature_dir, device, file_keypoints, rots,
            resize_to=CONFIG.params_sift_lightglue["resize_to"],
            detection_threshold=CONFIG.params_sift_lightglue["detection_threshold"],
            num_features=CONFIG.params_sift_lightglue["num_features"],
            min_matches=CONFIG.params_sift_lightglue["min_matches"],
        )
        gc.collect()
        files_keypoints.append(file_keypoints)
        timings['feature_matching'].append(t)

    if CONFIG.use_loftr:
        file_keypoints = f'{feature_dir}/matches_loftr_{CONFIG.params_loftr["resize_small_edge_to"]}pix.h5'
        t = detect_loftr(
            img_fnames, index_pairs, feature_dir, device, file_keypoints,
            resize_small_edge_to=CONFIG.params_loftr["resize_small_edge_to"],
            min_matches=CONFIG.params_loftr["min_matches"],
        )
        gc.collect()
        files_keypoints.append( file_keypoints )
        timings['feature_matching'].append(t)

    if CONFIG.use_dkm:
        file_keypoints = f'{feature_dir}/matches_dkm.h5'
        t = detect_dkm(
            img_fnames, index_pairs, feature_dir, device, file_keypoints,
            resize_to=CONFIG.params_dkm["resize_to"], 
            detection_threshold=CONFIG.params_dkm["detection_threshold"], 
            num_features=CONFIG.params_dkm["num_features"], 
            min_matches=CONFIG.params_dkm["min_matches"]
        )
        gc.collect()
        files_keypoints.append(file_keypoints)
        timings['feature_matching'].append(t)

    if CONFIG.use_matchformer:
        file_keypoints = f'{feature_dir}/matches_matchformer_{CONFIG.params_matchformer["resize_to"]}pix.h5'
        t = detect_matchformer(
            img_fnames, index_pairs, feature_dir, device, file_keypoints,
            resize_to=CONFIG.params_matchformer["resize_to"],
            num_features=CONFIG.params_matchformer["num_features"], 
            min_matches=CONFIG.params_matchformer["min_matches"]
        )
        gc.collect()
        files_keypoints.append( file_keypoints )
        timings['feature_matching'].append(t)

    #############################################################
    # merge keypoints
    #############################################################
    keypoints_merger(
        img_fnames,
        index_pairs,
        files_keypoints,
        feature_dir = feature_dir,
        filter_FundamentalMatrix = CONFIG.MERGE_PARAMS["filter_FundamentalMatrix"],
        filter_iterations = CONFIG.MERGE_PARAMS["filter_iterations"],
        filter_threshold = CONFIG.MERGE_PARAMS["filter_threshold"],
    )    
    return timings


def import_into_colmap(img_dir, feature_dir ='.featureout', database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

In [102]:
def reconstruct_from_db(feature_dir, img_dir):
    result = {}
    local_timings = {'RANSAC': [], 'Reconstruction': []}
    #############################################################
    # regist keypoints from h5 into colmap db
    #############################################################
    database_path = f'{feature_dir}/colmap.db'
    if os.path.isfile(database_path):
        os.remove(database_path)
    gc.collect()
    import_into_colmap(img_dir, feature_dir=feature_dir, database_path=database_path)
    output_path = f'{feature_dir}/colmap_rec'
    os.makedirs(output_path, exist_ok=True)
    print("colmap database")
    #############################################################
    # Calculate fundamental matrix with colmap api
    #############################################################
    t=time()
    # options = pycolmap.SiftMatchingOptions()
    # options.confidence = 0.9999
    # options.max_num_trials = 20000
    # pycolmap.match_exhaustive(database_path, sift_options=options)
    pycolmap.match_exhaustive(database_path)
    print("matching done!!!!")
    local_timings['RANSAC'].append(time() - t)
    print(f'RANSAC in {local_timings["RANSAC"][-1]:.4f} sec')

    #############################################################
    # Execute bundle adjustmnet with colmap api
    # --> Bundle adjustment Calcs Camera matrix, R and t
    #############################################################
    t=time()
    # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
    mapper_options = pycolmap.IncrementalPipelineOptions()
    # mapper_options.mapper.filter_max_reproj_error	 = 1.0
    # mapper_options.mapper.init_max_error = 2.0
    mapper_options.min_model_size = 5
    mapper_options.max_num_models = 25
    mapper_options.ba_global_images_freq = 5
    # mapper_options.ba_local_num_images = 8
    # mapper_options.mapper.abs_pose_min_inlier_ratio = 0.4
    mapper_options.ba_global_max_num_iterations = 100
    # mapper_options.mapper.filter_max_reproj_error = 6.0
    mapper_options.mapper.max_reg_trials = 10
    # mapper_options.mapper.init_min_num_inliers = 50
    mapper_options.mapper.abs_pose_min_num_inliers = 15
    

    
    maps = pycolmap.incremental_mapping(database_path=database_path, image_path=img_dir, 
                                        output_path=output_path, options=mapper_options)
    print(maps)
    for map_index, rec in maps.items():
        result[map_index] = {}
        for img_id, image in rec.images.items():
            result[map_index][image.name] = {
                'R': image.cam_from_world.rotation.matrix().tolist(),
                't': image.cam_from_world.translation.tolist()
            }
    # clear_output(wait=False)
    local_timings['Reconstruction'].append(time() - t)
    print(f'Reconstruction done in {local_timings["Reconstruction"][-1]:.4f} sec')

    #############################################################
    # Extract R,t from maps 
    #############################################################            
    return result, local_timings

In [103]:
# Collect vital info from the dataset

@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

# Set is_train=True to run the notebook on the training data.
# Set is_train=False if submitting an entry to the competition (test data is hidden, and different from what you see on the "test" folder).
is_train = True
data_dir = '/kaggle/input/image-matching-challenge-2025'
workdir = '/kaggle/working/result/'
os.makedirs(workdir, exist_ok=True)

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = {}
competition_data = pd.read_csv(sample_submission_csv)
for _, row in competition_data.iterrows():
    # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
    if row.dataset not in samples:
        samples[row.dataset] = []
    samples[row.dataset].append(
        Prediction(
            image_id=None if is_train else row.image_id,
            dataset=row.dataset,
            filename=row.image
        )
    )

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')

Dataset "imc2023_haiper" -> num_images=54
Dataset "imc2023_heritage" -> num_images=209
Dataset "imc2023_theather_imc2024_church" -> num_images=76
Dataset "imc2024_dioscuri_baalshamin" -> num_images=138
Dataset "imc2024_lizard_pond" -> num_images=214
Dataset "pt_brandenburg_british_buckingham" -> num_images=225
Dataset "pt_piazzasanmarco_grandplace" -> num_images=168
Dataset "pt_sacrecoeur_trevi_tajmahal" -> num_images=225
Dataset "pt_stpeters_stpauls" -> num_images=200
Dataset "amy_gardens" -> num_images=200
Dataset "fbk_vineyard" -> num_images=163
Dataset "ETs" -> num_images=22
Dataset "stairs" -> num_images=51


In [104]:
import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


def draw_keypoints_and_matches(images_input, unified_kp_path, remapped_matches_path, feature_dir='visualization_output'):
    output_dir = os.path.join(feature_dir, 'visualization_output')
    os.makedirs(output_dir, exist_ok=True)

    # Load images and determine image_keys for HDF5 lookup
    if isinstance(images_input[0], str):
        loaded_images = [cv2.imread(img_path) for img_path in images_input]
        image_keys = [os.path.basename(img_path) for img_path in images_input]
    else:
        loaded_images = images_input
        # If images_input are already arrays, you need to provide the corresponding keys
        # This part is crucial: image_keys MUST align with the HDF5 keys
        image_keys = image_keys_in_h5 # Use the predefined list for the dummy case

    # Load unified keypoints
    keypoints_data = {}
    with h5py.File(unified_kp_path, 'r') as f_kp:
        for img_name_raw in f_kp.keys():
            img_name = img_name_raw.decode('utf-8') if isinstance(img_name_raw, bytes) else img_name_raw
            keypoints_data[img_name] = f_kp[img_name_raw][()] # Access with raw key if bytes

    # Load remapped matches - CORRECTED LOGIC
    # Store (img1_key, img2_key) directly with matches for robust iteration
    matches_data_pairs = [] # Will store (img1_key, img2_key, matches_array)
    with h5py.File(remapped_matches_path, 'r') as f_matches:
        print("\n--- Loading remapped matches from HDF5 ---")
        for img1_group_key_candidate in tqdm(f_matches.keys(), desc="Loading matches"):
            img1_key = img1_group_key_candidate.decode('utf-8') if isinstance(img1_group_key_candidate, bytes) else img1_group_key_candidate

            img1_group = f_matches[img1_group_key_candidate] # Access with raw key

            if isinstance(img1_group, h5py.Group):
                for img2_dataset_key_candidate in img1_group.keys():
                    img2_key = img2_dataset_key_candidate.decode('utf-8') if isinstance(img2_dataset_key_candidate, bytes) else img2_dataset_key_candidate

                    try:
                        matches_array = img1_group[img2_dataset_key_candidate][()]
                        matches_data_pairs.append((img1_key, img2_key, matches_array))
                    except Exception as e:
                        print(f"Error loading matches for pair ({img1_key}, {img2_key}): {e}")
            else:
                print(f"Warning: Expected '{img1_key}' to be a group, but found {type(img1_group)}. Skipping its contents.")


    # --- Drawing Keypoints ---
    print("\n--- Drawing Keypoints ---")
    for i, img_key in enumerate(image_keys):
        if img_key in keypoints_data:
            img = loaded_images[i].copy()
            kpts = keypoints_data[img_key]

            for kp in kpts:
                x, y = int(kp[0]), int(kp[1])
                cv2.circle(img, (x, y), 3, (0, 255, 0), -1) # Green circle for keypoint

            output_kp_path = os.path.join(output_dir, f"keypoints_{img_key}")
            if len(img.shape) == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            cv2.imwrite(output_kp_path, img)
            print(f"Keypoints drawn on {img_key}, saved to {output_kp_path}")
        else:
            print(f"No keypoints found for {img_key} in unified keypoints file.")

    # --- Drawing Matches ---
    print("\n--- Drawing Matches ---")
    # Iterate through the (img1_key, img2_key, matches) tuples directly
    for img_name1, img_name2, matches in matches_data_pairs:
        # We no longer need to split img_pair_key, as we have img_name1 and img_name2 directly

        # Find the actual image objects and their keypoints using image_keys list
        try:
            img1_idx = image_keys.index(img_name1)
            img2_idx = image_keys.index(img_name2)
        except ValueError:
            print(f"Skipping matches for {img_name1}-{img_name2}: One or both image names not found in the provided 'images' list/keys.")
            continue

        img1 = loaded_images[img1_idx].copy()
        img2 = loaded_images[img2_idx].copy()

        kpts1 = keypoints_data.get(img_name1)
        kpts2 = keypoints_data.get(img_name2)

        if kpts1 is None or kpts2 is None:
            print(f"Skipping matches for {img_name1}-{img_name2}: keypoints not found for one or both images in unified keypoints.")
            continue
        if len(matches) == 0:
            print(f"No matches to draw for {img_name1}-{img_name2}.")
            continue

        # Ensure images are 3 channels for drawing lines
        if len(img1.shape) == 2:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) == 2:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

        # Create a concatenated image for drawing matches
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        max_h = max(h1, h2)
        matched_img = np.zeros((max_h, w1 + w2, 3), dtype=np.uint8)
        matched_img[0:h1, 0:w1] = img1
        matched_img[0:h2, w1:w1+w2] = img2

        num_matches_to_draw = min(len(matches), 200) # Draw up to 200 matches to avoid clutter, adjust as needed

        for i in range(num_matches_to_draw):
            match = matches[i]
            kp1_idx, kp2_idx = int(match[0]), int(match[1])

            # Bounds check for keypoint indices
            if kp1_idx >= len(kpts1) or kp2_idx >= len(kpts2):
                # print(f"Warning: Match index out of bounds for {img_name1}-{img_name2}. Skipping match {kp1_idx}-{kp2_idx}.")
                continue

            pt1 = tuple(map(int, kpts1[kp1_idx][:2]))
            pt2 = tuple(map(int, kpts2[kp2_idx][:2]))

            # Draw circles on the concatenated image
            cv2.circle(matched_img, pt1, 5, (0, 0, 255), 2) # Red circle on img1 side
            cv2.circle(matched_img, (pt2[0] + w1, pt2[1]), 5, (255, 0, 0), 2) # Blue circle on img2 side

            # Draw a line connecting the matched keypoints
            color = tuple(np.random.randint(0, 255, 3).tolist())
            cv2.line(matched_img, pt1, (pt2[0] + w1, pt2[1]), color, 1)

        output_match_path = os.path.join(output_dir, f"matches_{img_name1}_{img_name2}.png")
        cv2.imwrite(output_match_path, matched_img)
        print(f"Matches drawn between {img_name1} and {img_name2}, saved to {output_match_path}")


# Example call (replace with your actual 'images' list)
# If your 'images' are file paths:
# images_file_paths = ['path/to/your/image1.jpg', 'path/to/your/image2.jpg', ...]
# draw_keypoints_and_matches(images_file_paths, unified_kp_path, remapped_matches_path)

# If your 'images' are loaded numpy arrays (as in the dummy example above):
# draw_keypoints_and_matches(images, unified_kp_path, remapped_matches_path)

In [105]:
gc.collect()

max_images = None  # Used For debugging only. Set to None to disable.
datasets_to_process = None  # Not the best convention, but None means all datasets.

if is_train:
    # max_images = 5

    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	# 'amy_gardens',
    	# 'ETs',
    	# 'fbk_vineyard',
    	'stairs',
    	# Data from IMC 2023 and 2024.
    	# 'imc2024_dioscuri_baalshamin',
    	# 'imc2023_theather_imc2024_church',
    	# 'imc2023_heritage',
    	# 'imc2023_haiper',
    	# 'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	# 'pt_stpeters_stpauls',
    	# 'pt_brandenburg_british_buckingham',
    	# 'pt_piazzasanmarco_grandplace',
    	# 'pt_sacrecoeur_trevi_tajmahal',
    ]

timings = {
    'rotation_detection':[],
    "global feature extraction":[],
    "shortlisting":[],
    "feature_detection": [],
    "feature_matching":[],
    "RANSAC": [],
    "Reconstruction": [],
}
mapping_result_strs = []

# Load DINOv2 model (for feature extraction, not global descriptor here)
print("Loading DINOv2 model for patch feature extraction...")
dino_processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
dino_model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
dino_model = dino_model.eval().to(device)
print("DINOv2 model loaded.")

with concurrent.futures.ProcessPoolExecutor(max_workers=CONFIG.NUM_CORES) as executors:
    # print (f"Extracting on device {device}")
    for dataset, predictions in samples.items():
        if datasets_to_process and dataset not in datasets_to_process:
            print(f'Skipping "{dataset}"')
            continue
        
        images_dir = os.path.join(data_dir, 'train' if is_train else 'test', dataset)
        images = [os.path.join(images_dir, p.filename) for p in predictions]
        if max_images is not None:
            images = images[:max_images]
    
        print(f'\nProcessing dataset "{dataset}": {len(images)} images')
    
        filename_to_index = {p.filename: idx for idx, p in enumerate(predictions)}
    
        feature_dir = os.path.join(workdir, 'featureout', dataset)
        os.makedirs(feature_dir, exist_ok=True)
    
        # Wrap algos in try-except blocks so we can populate a submission even if one scene crashes.
        try:
            # --- Pipeline Execution ---
            
            #############################################################
            # get image rotations
            #############################################################
            t = time()
            # if CONFIG.ROTATION_CORRECTION:
            #     rots = exec_rotation_detection(images, device)
            # else:
            #     rots = [ 0 for fname in images ]
            rots = [ 0 for fname in images ]
            t = time()-t
            timings['rotation_detection'].append(t)
            print(f'rotation_detection for {len(images)} images : {t:.4f} sec')
            # print("!!!!!!!!!!!!!!!!!!!!!!")
            gc.collect()
            #############################################################
            # get image pairs
            #############################################################
            # 1. Detect ALIKED features and combine with DINO patch features
            t = time()
            index_pairs = get_image_pairs_shortlist(
                images,
                sim_th = 0.3, # should be strict
                min_pairs = 20, # we should select at least min_pairs PER IMAGE with biggest similarity
                max_pairs = 30,
                exhaustive_if_less = 60,
                device=device
            )
            timings['shortlisting'].append(time() - t)
            print (f'Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec')
            gc.collect()
            # print("\n--- Step 1: Detecting ALIKED and Combining with DINO Patch Features ---")
            # detect_aliked_and_combine_with_dino(
            #     img_fnames=images,
            #     feature_dir=feature_dir,
            #     num_features=4096,
            #     resize_to=1024,
            #     dino_processor=dino_processor,
            #     dino_model=dino_model,
            #     dino_patch_size=14, # Adjust based on your DINO model's patch size (e.g., 14 for DINOv2 base)
            #     device=device
            # )
            # timings['global feature extraction'].append(time() - t)
            # print (f'Gloabl feature extracting. Done in {time() - t:.4f} sec')
            # gc.collect()
            
            # # 2. Get image pairs shortlist using VLAD global descriptors
            # print("\n--- Step 2: Generating Image Pair Shortlist using VLAD ---")
            # # Adjust num_clusters_vlad as needed (e.g., 64, 128, 256)
            # # Higher clusters mean higher dimensionality for global descriptor.
            # index_pairs = get_image_pairs_shortlist_vlad(
            #     fnames=images,
            #     sim_th=0.5,
            #     min_pairs=20,
            #     exhaustive_if_less=20,
            #     feature_dir=feature_dir,
            #     num_clusters_vlad=128, # Example: 128 clusters for VLAD
            #     device=device
            # )
            # index_pairs = get_img_pairs_exhaustive(images)
            
            print(f"Generated {len(index_pairs)} image pairs using VLAD global descriptor.")
            timings['shortlisting'].append(time() - t)
            print (f'Shortlisting. Number of pairs to match: {len(index_pairs)}. Done in {time() - t:.4f} sec')
            gc.collect()
            #############################################################
            # get keypoints
            #############################################################    
            t=time()
            keypoints_timings = wrapper_keypoints(
                images, index_pairs, feature_dir, device, timings, rots
            )
            timings['feature_matching'] = keypoints_timings['feature_matching']
            gc.collect()
            print (f'Local feature extracting and matching. Done in {time() - t:.4f} sec')
            #############################################################
            # kick COLMAP reconstruction
            #############################################################            
            future = executors.submit(
                reconstruct_from_db, 
                feature_dir, images_dir)
            maps, local_timings = future.result()
            # 合并 timings（主进程里）
            for k in local_timings:
                timings[k].extend(local_timings[k])
            # clear_output(wait=False)
            registered = 0
            for map_index, cur_map in maps.items():  # cur_map: image_name → {'R': list, 't': list}
                for image_name, pose in cur_map.items():
                    idx = filename_to_index[image_name]
                    pred = predictions[idx]
                    pred.cluster_index = map_index
                    pred.rotation = np.array(pose['R'])  # convert back to np.ndarray
                    pred.translation = np.array(pose['t'])
                    registered += 1
            mapping_result_str = f"Dataset  {dataset} -> Registered {registered} / {len(images)} images with {len(maps)} clusters"
            mapping_result_strs.append(mapping_result_str)
            print(mapping_result_str)

            gc.collect()
        except Exception as e:
            print(e)
            # raise e
            mapping_result_str = f'Dataset "{dataset}" -> Failed!'
            mapping_result_strs.append(mapping_result_str)
            print(mapping_result_str)

print('\nResults')
for s in mapping_result_strs:
    print(s)

print('\nTimings')
for k, v in timings.items():
    print(f'{k} -> total={sum(v):.02f} sec.')

Loading DINOv2 model for patch feature extraction...
DINOv2 model loaded.
Skipping "imc2023_haiper"
Skipping "imc2023_heritage"
Skipping "imc2023_theather_imc2024_church"
Skipping "imc2024_dioscuri_baalshamin"
Skipping "imc2024_lizard_pond"
Skipping "pt_brandenburg_british_buckingham"
Skipping "pt_piazzasanmarco_grandplace"
Skipping "pt_sacrecoeur_trevi_tajmahal"
Skipping "pt_stpeters_stpauls"
Skipping "amy_gardens"
Skipping "fbk_vineyard"
Skipping "ETs"

Processing dataset "stairs": 51 images
rotation_detection for 51 images : 0.0000 sec
Shortlisting. Number of pairs to match: 1275. Done in 0.0003 sec
Generated 1275 image pairs using VLAD global descriptor.
Shortlisting. Number of pairs to match: 1275. Done in 0.2510 sec
aliked > rot_k=0, kpts.shape=torch.Size([632, 2]), descs.shape=torch.Size([632, 128])
aliked > rot_k=0, kpts.shape=torch.Size([970, 2]), descs.shape=torch.Size([970, 128])
aliked > rot_k=0, kpts.shape=torch.Size([440, 2]), descs.shape=torch.Size([440, 128])
aliked > r

  2%|▏         | 26/1275 [00:00<00:31, 39.69it/s]

aliked> stairs_split_1_1710453963274.png-stairs_split_1_1710453626698.png: 73 matches @ 1th pair(aliked+lightglue)
aliked> stairs_split_1_1710453963274.png-stairs_split_1_1710453643106.png: 133 matches @ 2th pair(aliked+lightglue)


  5%|▍         | 62/1275 [00:01<00:31, 38.86it/s]

aliked> stairs_split_1_1710453947066.png-stairs_split_1_1710453704934.png: 50 matches @ 3th pair(aliked+lightglue)
aliked> stairs_split_1_1710453947066.png-stairs_split_1_1710453675921.png: 83 matches @ 4th pair(aliked+lightglue)
aliked> stairs_split_1_1710453947066.png-stairs_split_1_1710453606287.png: 87 matches @ 5th pair(aliked+lightglue)
aliked> stairs_split_1_1710453947066.png-stairs_split_1_1710453990286.png: 224 matches @ 6th pair(aliked+lightglue)


...

100%|██████████| 1275/1275 [02:11<00:00,  9.70it/s]


Features matched in  144.6388 sec (disk+LightGlue)


filter_FundamentalMatrix: 258 matches --> 40 matches
stairs_split_1_1710453963274.png-stairs_split_1_1710453626698.png: 258 --> 40 matches
filter_FundamentalMatrix: 798 matches --> 106 matches
stairs_split_1_1710453963274.png-stairs_split_1_1710453643106.png: 798 --> 106 matches
skipped key1=stairs_split_1_1710453947066.png, key2=stairs_split_1_1710453930259.png: mkpts.shape=(14, 4) after filtered.
skipped key1=stairs_split_1_1710453947066.png, key2=stairs_split_1_1710453704934.png: mkpts.shape=(8, 4) after filtered.
filter_FundamentalMatrix: 380 matches --> 25 matches
stairs_split_1_1710453947066.png-stairs_split_1_1710453675921.png: 380 --> 25 matches
filter_FundamentalMatrix: 139 matches --> 24 matches
stairs_split_1_1710453947066.png-stairs_split_1_1710453606287.png: 139 --> 24 matches
filter_FundamentalMatrix: 496 matches --> 47 matches
stairs_split_1_1710453947066.png-stairs_split_1_1710453990286.png: 496 --> 47 matches
skipped key1=stairs_split_1_1710453947066.png, key2=stairs_s

 17%|█▋        | 142/820 [00:00<00:00, 4396.58it/s]


colmap database
matching done!!!!
RANSAC in 0.5834 sec
{0: Reconstruction(num_reg_images=3, num_cameras=3, num_points3D=44, num_observations=94)}
Reconstruction done in 0.8190 sec
Dataset  stairs -> Registered 3 / 51 images with 1 clusters

Results
Dataset  stairs -> Registered 3 / 51 images with 1 clusters

Timings
rotation_detection -> total=0.00 sec.
global feature extraction -> total=0.00 sec.
shortlisting -> total=0.25 sec.
feature_detection -> total=0.00 sec.
feature_matching -> total=227.28 sec.
RANSAC -> total=0.58 sec.
Reconstruction -> total=0.82 sec.


In [106]:
# Helpers
array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
none_to_str = lambda n: ';'.join(['nan'] * n)

submission_file = '/kaggle/working/submission.csv'
with open(submission_file, 'w') as f:
    if is_train:
        f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset, predictions in samples.items():
            for prediction in predictions:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'

                # ✅ `rotation` is a list of lists, flatten it
                if prediction.rotation is None:
                    rotation_str = none_to_str(9)
                else:
                    rotation_flat =  prediction.rotation.flatten()  # flatten 3x3 list -> 9 elems
                    rotation_str = array_to_str(rotation_flat)

                # ✅ `translation` is a flat list
                if prediction.translation is None:
                    translation_str = none_to_str(3)
                else:
                    translation_str = array_to_str(prediction.translation)

                f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation_str},{translation_str}\n')
    else:
        f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
        for dataset, predictions in samples.items():
            for prediction in predictions:
                cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'

                if prediction.rotation is None:
                    rotation_str = none_to_str(9)
                else:
                    rotation_flat =  prediction.rotation.flatten()
                    rotation_str = array_to_str(rotation_flat)

                if prediction.translation is None:
                    translation_str = none_to_str(3)
                else:
                    translation_str = array_to_str(prediction.translation)

                f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation_str},{translation_str}\n')

# Preview the output
!head {submission_file}


dataset,scene,image,rotation_matrix,translation_vector
imc2023_haiper,outliers,fountain_image_116.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_108.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_101.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_082.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_071.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_025.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_000.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_007.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan
imc2023_haiper,outliers,fountain_image_012.png,nan;nan;nan;nan;nan;nan;nan;nan;nan,nan;nan;nan


In [107]:
# Definitely Compute results if running on the training set.
# Do not do this when submitting a notebook for scoring. All you have to do is save your submission to /kaggle/working/submission.csv.

if is_train:
    t = time()
    final_score, dataset_scores = metric.score(
        gt_csv='/kaggle/input/image-matching-challenge-2025/train_labels.csv',
        user_csv=submission_file,
        thresholds_csv='/kaggle/input/image-matching-challenge-2025/train_thresholds.csv',
        mask_csv=None if is_train else os.path.join(data_dir, 'mask.csv'),
        inl_cf=0,
        strict_cf=-1,
        verbose=True,
    )
    print(f'Computed metric in: {time() - t:.02f} sec.')

imc2023_haiper: score=0.00% (mAA=0.00%, clusterness=100.00%)
imc2023_heritage: score=0.00% (mAA=0.00%, clusterness=100.00%)
imc2023_theather_imc2024_church: score=0.00% (mAA=0.00%, clusterness=100.00%)
imc2024_dioscuri_baalshamin: score=0.00% (mAA=0.00%, clusterness=100.00%)
imc2024_lizard_pond: score=0.00% (mAA=0.00%, clusterness=100.00%)
pt_brandenburg_british_buckingham: score=0.00% (mAA=0.00%, clusterness=100.00%)
pt_piazzasanmarco_grandplace: score=0.00% (mAA=0.00%, clusterness=100.00%)
pt_sacrecoeur_trevi_tajmahal: score=0.00% (mAA=0.00%, clusterness=100.00%)
pt_stpeters_stpauls: score=0.00% (mAA=0.00%, clusterness=100.00%)
amy_gardens: score=0.00% (mAA=0.00%, clusterness=100.00%)
fbk_vineyard: score=0.00% (mAA=0.00%, clusterness=100.00%)
ETs: score=0.00% (mAA=0.00%, clusterness=100.00%)
stairs: score=0.00% (mAA=0.00%, clusterness=50.00%)
Average over all datasets: score=0.00% (mAA=0.00%, clusterness=96.15%)
Computed metric in: 0.06 sec.
